# 데이터 작업하기

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Dataset: sample과 label을 저장

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:00<00:00, 115604052.22it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 7385007.61it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 60591421.73it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 18550066.14it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

# DataLoader: Dataset을 순회 가능한 객체로 감쌈
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


---

# 모델 만들기

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"                              # ARM 맥북 GPU 가속기 설정
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)                 # 2차원 입력을 1차원으로 Flatten()
        logits = self.linear_relu_stack(x)  # 사전에 정의한 모델인 Sequential로 입력
        return logits

model = NeuralNetwork().to(device)          # NeuralNetwork를 device로 보냄
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


---

# 모델 매개변수 최적화하기

In [7]:
# loss_fn: CrossEntropy / optimizer: Stochastic Gradient Descent, lr=0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)                  # 데이터셋 크기 확인
    for batch, (X, y) in enumerate(dataloader):     # 배치별로 학습 진행
        X, y = X.to(device), y.to(device)           # device로 데이터 이동

        pred = model(X)                             # tf의 y_pred
        loss = loss_fn(pred, y)                     # loss_fn에 pred와 label y를 넣고 비교

        optimizer.zero_grad()                       # 이전에 계산된 경사값을 초기화
        loss.backward()                             # 오류 역전파 계산
        optimizer.step()                            # SGD로 매개변수 업데이트

        # 매 100번째 배치마다 현재 loss와 처리된 데이터 개수 current를 출력
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()                                    # 모델 평가 모드 설정
    test_loss, correct = 0, 0                       # loss와 정확히 예측한 correct
    with torch.no_grad():                           # 그래디언트 계산 비활성화
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            # 정확한 예측 개수 계산
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303774  [   64/60000]
loss: 2.290728  [ 6464/60000]
loss: 2.272647  [12864/60000]
loss: 2.273123  [19264/60000]
loss: 2.260914  [25664/60000]
loss: 2.228867  [32064/60000]
loss: 2.245543  [38464/60000]
loss: 2.208960  [44864/60000]
loss: 2.206449  [51264/60000]
loss: 2.184839  [57664/60000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 2.174990 

Epoch 2
-------------------------------
loss: 2.180228  [   64/60000]
loss: 2.171994  [ 6464/60000]
loss: 2.128336  [12864/60000]
loss: 2.144727  [19264/60000]
loss: 2.102773  [25664/60000]
loss: 2.040496  [32064/60000]
loss: 2.072388  [38464/60000]
loss: 2.000027  [44864/60000]
loss: 2.007370  [51264/60000]
loss: 1.942954  [57664/60000]
Test Error: 
 Accuracy: 56.5%, Avg loss: 1.940857 

Epoch 3
-------------------------------
loss: 1.968301  [   64/60000]
loss: 1.942537  [ 6464/60000]
loss: 1.845609  [12864/60000]
loss: 1.880448  [19264/60000]
loss: 1.781511  [25664/60000]
loss: 1.718245  [32064/600

---

# 모델 저장하기

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# 모델 불러오기

In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
